In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# Load your cleaned dataset (without 'date' column)
df = pd.read_csv("data.csv")  # Replace with your CSV filename

# Features and target
X = df[['temperature', 'humidity']].values
y = df['label'].values  # Make sure you have a column 'label' as 0/1 for anomaly

# Normalize data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Split into train, val, test sets (70% train, 15% val, 15% test)
X_temp, X_test, y_temp, y_test = train_test_split(X_scaled, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1765, random_state=42)

In [2]:
import tensorflow as tf
import numpy as np

# Load the trained model
model = tf.keras.models.load_model("base_model.h5")

# Define representative dataset for quantization
def representative_dataset():
    for i in range(100):
        # Assuming X_train is in memory and normalized to [0, 1]
        data = X_train[i].reshape(1, X_train.shape[1])
        yield [tf.convert_to_tensor(data, dtype=tf.float32)]

# Convert model to fully quantized int8 TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

# Convert and save
quantized_tflite_model = converter.convert()

with open("tinyml_anomaly_model_quant.tflite", "wb") as f:
    f.write(quantized_tflite_model)

print("✅ Quantized model saved as 'tinyml_anomaly_model_quant.tflite'")


INFO:tensorflow:Assets written to: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpazht012g/assets


INFO:tensorflow:Assets written to: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpazht012g/assets
/Users/oscarpatrikminj/Documents/IITR/Working/dep/venv/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2025-06-11 22:33:34.407509: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-06-11 22:33:34.407526: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-06-11 22:33:34.408126: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/bs/x0lj933d1hv0py0d4w2ypdp40000gn/T/tmpazht012g
2025-06-11 22:33:34.408878: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-06-11 22:33:34.408887: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /var/folders/bs/

✅ Quantized model saved as 'tinyml_anomaly_model_quant.tflite'


fully_quantize: 0, inference_type: 6, input_inference_type: UINT8, output_inference_type: UINT8


In [8]:
import os


quant_path = "tinyml_anomaly_model_quant.tflite"
quant_size_kb = os.path.getsize(quant_path) / 1024
print(f"📦 Quantized TFLite Model Size: {quant_size_kb:.2f} KB")


📦 Quantized TFLite Model Size: 2.87 KB


In [5]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path="quant.tflite")
interpreter.allocate_tensors()

# Get input/output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Run inference on test data
y_pred = []
for sample in X_test:  # Make sure X_test is scaled
    # Quantize input
    scale, zero_point = input_details[0]['quantization']
    input_data = np.array([sample], dtype=np.float32)
    input_data_uint8 = np.round(input_data / scale + zero_point).astype(np.uint8)

    interpreter.set_tensor(input_details[0]['index'], input_data_uint8)
    interpreter.invoke()

    # Dequantize output if needed
    output_data = interpreter.get_tensor(output_details[0]['index'])
    output_scale, output_zero_point = output_details[0]['quantization']
    output = output_scale * (output_data.astype(np.float32) - output_zero_point)
    output = output[0][0]
    y_pred.append(1 if output > 0.5 else 0)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ TFLite Model Test Accuracy: {accuracy * 100:.2f}%")


✅ TFLite Model Test Accuracy: 86.67%
